In [1]:
import pandas as pd
import docker
import os
import re
import json

In [16]:
#sol文件list，
def run_smartbugs(sol_list,root_dir,tools,save_addr):
    for tool in tools:
        for i in range(3):
            text = os.popen("python3 smartBugs.py --tool {} --file {}".format(tool,os.path.join(root_dir,sol_list[i]))).read()
            print(str(text))
            print(os.path.join(root_dir,sol_list[i]))



In [3]:
def run_smartbugs2(sol_list,root_dir,tools,save_addr):
    for tool in tools:#工具list
        for i in range(len(sol_list)):#sol文件名称list，在后面才转成地址
            text = os.popen("python3 smartBugs.py --tool {} --file {}".format(tool,os.path.join(root_dir,sol_list[i]))).read()
            print(str(text))#看结果
            print(os.path.join(root_dir,ans[i]))#显示地址



In [4]:
#append_contract_data
append_contract = pd.read_csv('append_contract_data.csv')
sni_contract = pd.read_csv('snippet_contract_v2.csv')
con_match = pd.read_csv('contract_match_true.csv')
code_contract = pd.read_csv('16w_contract_v2.csv').drop_duplicates(['dealed_code'])

code_function = pd.read_csv('16w_function_v2.csv')
sni_function = pd.read_csv('snippet_function_v2.csv')
fun_match = pd.read_csv('function_match_v2.csv')
seg0 = pd.read_csv("segment_0.9.csv")
seg1 = pd.read_csv("segment_0.8.csv")


In [5]:
seg1

,sni_index,code_index,sni,bleu,code,check
0,8,27045,function is\n\n function sendValue(address ...,0.859892,"function safeEthTransfer(address recipient, ui...",3
1,13,46541,function () payable public{\n mintable...,0.822624,function() public payable stoppable {\n ...,3
2,38,47107,"function safeTransfer(ERC223Interface token, a...",0.827878,function safeTransfer(\n\n IERC20 _toke...,1
3,39,34900,function safeTransferFrom(ERC223Interface toke...,0.816338,"function safeTransfer(\n ERC20 token,\n...",3
4,41,1861,function safeIncreaseAllowance(ERC223Interface...,0.816464,"function safeDecreaseAllowance(IERC20 token, a...",3
...,...,...,...,...,...,...
186,5676,49479,"function validPurchase (address beneficiary,ui...",0.833912,function _preValidatePurchase(\n\n addr...,3
187,5696,25778,function LearnToken (uint256 _initialSupply) p...,0.830782,function DelphiToken() public {\n uint2...,3
188,5701,50308,function() public payable {\n enterICO();\n},0.883180,function enter() payable public{\n\n },3
189,5888,114970,function censorChatMessage(uint256 messageInde...,0.813743,function censorChatMessage(uint256 chatMessage...,1


In [6]:
temp_tt = code_contract.iloc[con_match[con_match['bleu'] >= 0.9]['code']][['code_name','code']]
temp_tt

,code_name,code
44,0xcf2b19384ea3f027f8620db3f797ae61bb475125.sol,contract multiowned {\n\n\n\n \n\n\n\n \...
45,0xcf2b19384ea3f027f8620db3f797ae61bb475125.sol,contract daylimit is multiowned {\n\n\n\n \...
46,0xcf2b19384ea3f027f8620db3f797ae61bb475125.sol,contract multisig {\n\n\n\n \n\n\n\n \n\...
21772,0x6d056af97b52e343ce646fa79cf09ca735ff8db5.sol,contract MomsAvenueToken {\n function trans...
7762,0x539699ece91480179f9a62d842d0db49ee509b4b.sol,contract KKKKToken{\n\n uint256 public tota...
...,...,...
7324,0x6ba2fa9f8ad7b7c376210c0fb4fd311a66d98e8e.sol,contract Bakery {\n\n\n\n \n\n\n\n address[]...
56570,0x937a7486f5eb354c4a77882542333c4e8919203a.sol,contract SendToMany\n{\n address[] public r...
2076,0x4333defa1bf3b49246666d1dea395f98e816a032.sol,contract BitQueen is ERC20 {\n\n string publi...
2508,0x3F937666359555684192d7445D995fF358F02E21.sol,contract ContractReceiver {\n function toke...


In [8]:
def func(temp_tt):
    ans = []
    tete = list(temp_tt['code'])
    for i in tete:
        temptt = i.split()[1]
        if(temptt[-1] == '{'):
            temptt = temptt[:-1]
        ans.append(temptt)
    df_new = pd.DataFrame({
        'address':temp_tt['code_name'],
        'name':ans
    })
    return list(temp_tt['code_name']),ans
name,code = func(temp_tt)
name2,code2 = func(append_contract[['code_name','code']])
name+=name2
code+=code2
new_df = pd.DataFrame({
    'address':name,
    'name':code
})
new_df#862 contract

,address,name
0,0xcf2b19384ea3f027f8620db3f797ae61bb475125.sol,multiowned
1,0xcf2b19384ea3f027f8620db3f797ae61bb475125.sol,daylimit
2,0xcf2b19384ea3f027f8620db3f797ae61bb475125.sol,multisig
3,0x6d056af97b52e343ce646fa79cf09ca735ff8db5.sol,MomsAvenueToken
4,0x539699ece91480179f9a62d842d0db49ee509b4b.sol,KKKKToken
...,...,...
858,0x7b9e81ec3591b570aa2c9fbec77f7dda76ac4f6d.sol,Eater
859,0x3387c42ab6d4919c669d41cc27eb2e3fc714502f.sol,Now
860,0x3b9dc0e353b7bc689893a87cd697f08e06feee81.sol,BlankContract
861,0xa1b93fe007a6906027659e38cbd92e02bbfe30f6.sol,NGNT


In [9]:
append_contract[['code_name','code']]

,code_name,code
0,0x59f243d056ada87eb563c2d507fe4822f8ccccac.sol,contract MSD4 {\n\n \n\n function greenOn...
1,0x90e76c2c45599443339eca66bec2d05f20bb6f12.sol,contract Ownable {\n address public owner;\n\...
2,0x3e8b42754c8eafdc8e732f4346126e920ede8548.sol,contract HelloWorld{\n\n function Hello() p...
3,0x2bbca37e5fd308c406fa4d9a07897fc12df0afef.sol,contract Faucet {\n\n \n\n \n\n funct...
4,0x60e4c1f409cc41309a92e8da2a24b1180888d0c3.sol,contract ERC20Token {\n\n function transfer...
...,...,...
240,0x7b9e81ec3591b570aa2c9fbec77f7dda76ac4f6d.sol,contract Eater{\n\tfunction(){\n\t\t\n\t}\n}
241,0x3387c42ab6d4919c669d41cc27eb2e3fc714502f.sol,contract Now {\n\n \n\n function getNow() pu...
242,0x3b9dc0e353b7bc689893a87cd697f08e06feee81.sol,contract BlankContract {\n constructor() pu...
243,0xa1b93fe007a6906027659e38cbd92e02bbfe30f6.sol,contract NGNT is V1 {\n\n}


In [11]:
temp_618 = code_contract.iloc[con_match[con_match['bleu'] >= 0.9]['code']]
na = list(temp_618['code_name']) + list(append_contract['code_name'])
len(na)

863

In [20]:
temp_idx = list(code_contract.iloc[con_match[con_match['bleu'] >= 0.9]['code']].index) + list(append_contract['code_index'])
temp_idx

[44,
 45,
 46,
 21772,
 7762,
 716,
 2847,
 38100,
 108,
 310,
 51,
 110,
 453,
 40664,
 109,
 46969,
 15,
 66,
 440,
 716,
 875,
 44206,
 44207,
 2889,
 236,
 44208,
 44209,
 33,
 91,
 448,
 24,
 115,
 1603,
 657,
 18,
 33,
 215,
 91,
 935,
 54968,
 1908,
 4,
 13164,
 4667,
 17730,
 0,
 160,
 16,
 36124,
 404,
 4830,
 89,
 527,
 1902,
 116,
 118,
 7273,
 881,
 2215,
 110,
 1531,
 56827,
 42469,
 16379,
 16381,
 2285,
 57036,
 22604,
 4381,
 11876,
 2411,
 65045,
 56430,
 47746,
 102,
 1933,
 540,
 1934,
 23815,
 8957,
 11876,
 53109,
 359,
 1414,
 159,
 160,
 16,
 162,
 163,
 164,
 259,
 1039,
 320,
 258,
 4805,
 884,
 24234,
 2698,
 3812,
 866,
 6929,
 108,
 0,
 82,
 98,
 18,
 26294,
 17730,
 448,
 22629,
 17695,
 24,
 3462,
 56875,
 56876,
 451,
 0,
 13573,
 375,
 23,
 29969,
 2846,
 2847,
 0,
 10224,
 47801,
 3087,
 440,
 224,
 225,
 1042,
 54968,
 114,
 976,
 977,
 197,
 978,
 6969,
 259,
 320,
 322,
 1552,
 3359,
 20883,
 966,
 69,
 100,
 69,
 10148,
 10388,
 1502,
 12786,
 2154,

In [21]:
len(temp_idx)

863

In [12]:
tttt = pd.Series(list(append_contract['code']) + list(code_contract.iloc[con_match[con_match['bleu'] >= 0.9]['code']]['code']))
tttt = tttt.drop_duplicates()
tttt

0      contract MSD4 {\n\n   \n\n    function greenOn...
1      contract Ownable {\n  address public owner;\n\...
2      contract HelloWorld{\n\n    function Hello() p...
3      contract Faucet {\n\n    \n\n    \n\n    funct...
4      contract ERC20Token {\n\n    function transfer...
                             ...                        
858    contract Bakery {\n\n\n\n  \n\n\n\n  address[]...
859    contract SendToMany\n{\n    address[] public r...
860    contract BitQueen is ERC20 {\n\n  string publi...
861    contract ContractReceiver {\n    function toke...
862    contract test { function multiply(uint a) publ...
Length: 633, dtype: object

,0
0,contract MSD4 {\n\n \n\n function greenOn...
1,contract Ownable {\n address public owner;\n\...
2,contract HelloWorld{\n\n function Hello() p...
3,contract Faucet {\n\n \n\n \n\n funct...
4,contract ERC20Token {\n\n function transfer...
...,...
628,contract Bakery {\n\n\n\n \n\n\n\n address[]...
629,contract SendToMany\n{\n address[] public r...
630,contract BitQueen is ERC20 {\n\n string publi...
631,contract ContractReceiver {\n function toke...


In [17]:
func_index = seg0[seg0['check'] <= 1]['code_index'].append(seg1[seg1['check'] <= 1]['code_index']).append(fun_match[fun_match['bleu'] == 1]['code']).drop_duplicates()
func_code = code_function.iloc[func_index]['code_name']
#现在将bleu>=0.9和append放在一起，总共566个完整合约，对应900+匹配对
bleu_root = os.path.join("data_sol")
blue9AndAppend = list((code_contract.iloc[con_match[con_match['bleu'] >= 0.9]['code']]['code_name'].append(append_contract['code_name'])).drop_duplicates())
#所有sni_code的地址
root_dir = os.path.join("paper","sni_code")
sol_list = []
for index,row in sni_contract.iterrows():   
    code = row['code']
    sol_list.append('{}.sol'.format(index))

In [58]:
len(blue9AndAppend)
#multi = pd.read_csv('68_code_name.csv')
#multi = multi['code_name']
len(multi)
len(func_code)
sol_name_temp = list(blue9AndAppend) + list(multi) + list(func_code)
sol_name_temp
ttt = pd.Series(sol_name_temp).drop_duplicates()
ttt
ttt = ttt.str.slice(stop=-4)
#ttt.to_csv("all_complete_contract_address.csv")

In [63]:
multi= pd.read_csv('68_code_name.csv')
multi

,Unnamed: 0,code_name
0,74391,0x36a715c6572bf1511647aa25a9903366d7126946.sol
1,53263,0x0515023dc5ab2a88713ab5a03011e56ea754ad6f.sol
2,121813,0xadfc7b570f460ee2c9983fbb9c7b49667309be12.sol
3,14522,0x9B3470174fEBC7e3d7B55B6c562dEE1366dBDd78.sol
4,2637,0x75a88d4b4756aeb2438a1a7cd283d218e3ddf4de.sol
...,...,...
61,104102,0x5030f2fc18658a6de748cb23cbac9b2df2595618.sol
62,66552,0xbc7942054f77b82e8a71ace170e4b00ebae67eb6.sol
63,49378,0xff4b417892f3ca1678a288cc6296a7094bf98f0b.sol
64,73448,0xBfcC619F27f04DdA35277888BCD673c3CE3C0dc3.sol


In [ ]:
#run
run_smartbugs(list(func_code),bleu_root,['slither','mythril','oyente','osiris'],'bleu_func_detection')

In [11]:
#multi_68
multi = pd.read_csv('68_code_name.csv')
multi = multi['code_name']
bleu_root = os.path.join("data_sol")

In [56]:
multi = pd.read_csv('68_code_name.csv')
multi

,Unnamed: 0,code_name
0,74391,0x36a715c6572bf1511647aa25a9903366d7126946.sol
1,53263,0x0515023dc5ab2a88713ab5a03011e56ea754ad6f.sol
2,121813,0xadfc7b570f460ee2c9983fbb9c7b49667309be12.sol
3,14522,0x9B3470174fEBC7e3d7B55B6c562dEE1366dBDd78.sol
4,2637,0x75a88d4b4756aeb2438a1a7cd283d218e3ddf4de.sol
...,...,...
61,104102,0x5030f2fc18658a6de748cb23cbac9b2df2595618.sol
62,66552,0xbc7942054f77b82e8a71ace170e4b00ebae67eb6.sol
63,49378,0xff4b417892f3ca1678a288cc6296a7094bf98f0b.sol
64,73448,0xBfcC619F27f04DdA35277888BCD673c3CE3C0dc3.sol


In [18]:
run_smartbugs(sol_list,root_dir,['slither','mythril','oyente','osiris'],'sni_code_detection')



~/paper/sni_code/0.sol

~/paper/sni_code/1.sol

~/paper/sni_code/2.sol

~/paper/sni_code/0.sol

~/paper/sni_code/1.sol

~/paper/sni_code/2.sol

~/paper/sni_code/0.sol

~/paper/sni_code/1.sol

~/paper/sni_code/2.sol

~/paper/sni_code/0.sol

~/paper/sni_code/1.sol

~/paper/sni_code/2.sol


别运行-》

7161
stackflow_sol_categorized/question/contract/7089-1.sol
stackflow_sol_categorized/question/contract/7089-1.sol
stackflow_sol_categorized/question/contract/7089-1.sol
stackflow_sol_categorized/question/contract/27167-0.sol
stackflow_sol_categorized/question/contract/33837-0.sol
stackflow_sol_categorized/question/contract/10078-0.sol
stackflow_sol_categorized/question/contract/9000-0.sol
stackflow_sol_categorized/question/contract/4029-0.sol
stackflow_sol_categorized/question/contract/37927-0.sol
stackflow_sol_categorized/question/contract/37927-0.sol
stackflow_sol_categorized/question/contract/70551-1.sol
stackflow_sol_categorized/question/contract/70551-1.sol
stackflow_sol_categorized/question/contract/29245-0.sol
stackflow_sol_categorized/question/contract/29245-0.sol
stackflow_sol_categorized/question/contract/65048-0.sol
stackflow_sol_categorized/question/contract/66537-0.sol
stackflow_sol_categorized/question/contract/66537-0.sol
stackflow_sol_categorized/question/contract/5012

stackflow_sol_categorized/question/contract/34687-0.sol
stackflow_sol_categorized/question/contract/34687-0.sol
stackflow_sol_categorized/question/contract/70310-0.sol
stackflow_sol_categorized/question/contract/70310-0.sol
stackflow_sol_categorized/question/contract/60478-1.sol
stackflow_sol_categorized/question/contract/47910-0.sol
stackflow_sol_categorized/question/contract/37280-0.sol
stackflow_sol_categorized/question/contract/37280-0.sol
stackflow_sol_categorized/question/contract/72417-0.sol
stackflow_sol_categorized/question/contract/72417-0.sol
stackflow_sol_categorized/question/contract/64480-0.sol
stackflow_sol_categorized/question/contract/72435-0.sol
stackflow_sol_categorized/question/contract/46498-1.sol
stackflow_sol_categorized/question/contract/73901-0.sol
stackflow_sol_categorized/question/contract/28573-0.sol
stackflow_sol_categorized/question/contract/30371-1.sol
stackflow_sol_categorized/question/contract/68818-0.sol
stackflow_sol_categorized/question/contract/4260

stackflow_sol_categorized/question/contract/69902-0.sol
stackflow_sol_categorized/question/contract/27117-1.sol
stackflow_sol_categorized/question/contract/40290-0.sol
stackflow_sol_categorized/question/contract/57592-0.sol
stackflow_sol_categorized/question/contract/73530-0.sol
stackflow_sol_categorized/question/contract/73530-0.sol
stackflow_sol_categorized/question/contract/73530-0.sol
stackflow_sol_categorized/question/contract/73530-0.sol
stackflow_sol_categorized/question/contract/78970-1.sol
stackflow_sol_categorized/question/contract/78970-1.sol
stackflow_sol_categorized/question/contract/80529-0.sol
stackflow_sol_categorized/question/contract/31530-1.sol
stackflow_sol_categorized/question/contract/38371-0.sol
stackflow_sol_categorized/question/contract/38371-0.sol
stackflow_sol_categorized/question/contract/64736-0.sol
stackflow_sol_categorized/question/contract/81999-1.sol
stackflow_sol_categorized/question/contract/52133-0.sol
stackflow_sol_categorized/question/contract/3401

stackflow_sol_categorized/question/contract/80533-0.sol
stackflow_sol_categorized/question/contract/27085-1.sol
stackflow_sol_categorized/question/contract/27085-1.sol
stackflow_sol_categorized/question/contract/56575-0.sol
stackflow_sol_categorized/question/contract/56575-0.sol
stackflow_sol_categorized/question/contract/56575-0.sol
stackflow_sol_categorized/question/contract/54802-0.sol
stackflow_sol_categorized/question/contract/9741-1.sol
stackflow_sol_categorized/question/contract/9741-1.sol
stackflow_sol_categorized/question/contract/70010-2.sol
stackflow_sol_categorized/question/contract/3512-0.sol
stackflow_sol_categorized/question/contract/42494-0.sol
stackflow_sol_categorized/question/contract/37441-0.sol
stackflow_sol_categorized/question/contract/40948-2.sol
stackflow_sol_categorized/question/contract/74015-0.sol
stackflow_sol_categorized/question/contract/73740-0.sol
stackflow_sol_categorized/question/contract/73740-0.sol
stackflow_sol_categorized/question/contract/73740-0

stackflow_sol_categorized/question/other/68887-0.sol
stackflow_sol_categorized/question/other/56879-0.sol
stackflow_sol_categorized/question/other/83328-1.sol
stackflow_sol_categorized/question/other/22929-0.sol
stackflow_sol_categorized/question/other/46366-0.sol
stackflow_sol_categorized/question/other/69891-0.sol
stackflow_sol_categorized/question/other/58838-0.sol
stackflow_sol_categorized/question/other/61723-0.sol
stackflow_sol_categorized/question/other/61723-0.sol
stackflow_sol_categorized/question/other/47567-0.sol
stackflow_sol_categorized/question/other/47567-0.sol
stackflow_sol_categorized/question/other/47567-0.sol
stackflow_sol_categorized/question/other/62859-0.sol
stackflow_sol_categorized/question/other/62859-0.sol
stackflow_sol_categorized/question/other/15747-2.sol
stackflow_sol_categorized/question/other/82461-0.sol
stackflow_sol_categorized/question/other/69235-1.sol
stackflow_sol_categorized/question/other/2624-0.sol
stackflow_sol_categorized/question/other/71873-

stackflow_sol_categorized/answer/contract/42715-0-2.sol
stackflow_sol_categorized/answer/contract/24160-0-0.sol
stackflow_sol_categorized/answer/contract/54752-0-0.sol
stackflow_sol_categorized/answer/contract/76673-1-0.sol
stackflow_sol_categorized/answer/contract/71609-2-1.sol
stackflow_sol_categorized/answer/contract/73611-1-0.sol
stackflow_sol_categorized/answer/contract/73611-1-0.sol
stackflow_sol_categorized/answer/contract/65714-0-0.sol
stackflow_sol_categorized/answer/contract/58432-0-1.sol
stackflow_sol_categorized/answer/contract/3285-0-1.sol
stackflow_sol_categorized/answer/contract/66610-3-0.sol
stackflow_sol_categorized/answer/contract/2-4-0.sol
stackflow_sol_categorized/answer/contract/62375-0-0.sol
stackflow_sol_categorized/answer/contract/62448-0-0.sol
stackflow_sol_categorized/answer/contract/10878-0-0.sol
stackflow_sol_categorized/answer/contract/10878-0-0.sol
stackflow_sol_categorized/answer/contract/56475-2-0.sol
stackflow_sol_categorized/answer/contract/2567-0-0.so

stackflow_sol_categorized/answer/contract/43988-0-0.sol
stackflow_sol_categorized/answer/contract/28680-0-0.sol
stackflow_sol_categorized/answer/contract/43931-1-0.sol
stackflow_sol_categorized/answer/contract/80183-0-0.sol
stackflow_sol_categorized/answer/contract/82736-0-0.sol
stackflow_sol_categorized/answer/contract/82736-0-0.sol
stackflow_sol_categorized/answer/contract/65733-0-0.sol
stackflow_sol_categorized/answer/contract/20834-0-1.sol
stackflow_sol_categorized/answer/contract/20834-0-1.sol
stackflow_sol_categorized/answer/contract/47894-0-0.sol
stackflow_sol_categorized/answer/contract/35909-0-1.sol
stackflow_sol_categorized/answer/contract/2450-0-0.sol
stackflow_sol_categorized/answer/contract/41874-1-0.sol
stackflow_sol_categorized/answer/contract/11593-1-0.sol
stackflow_sol_categorized/answer/contract/44247-0-0.sol
stackflow_sol_categorized/answer/contract/44247-0-0.sol
stackflow_sol_categorized/answer/contract/38968-0-0.sol
stackflow_sol_categorized/answer/contract/40430-0

stackflow_sol_categorized/answer/other/39259-0-1.sol
stackflow_sol_categorized/answer/other/11383-0-0.sol
stackflow_sol_categorized/answer/other/63092-1-0.sol
stackflow_sol_categorized/answer/other/63377-0-0.sol
stackflow_sol_categorized/answer/other/53243-0-0.sol
stackflow_sol_categorized/answer/other/63712-0-0.sol
stackflow_sol_categorized/answer/other/46268-2-0.sol
stackflow_sol_categorized/answer/other/24003-0-0.sol
stackflow_sol_categorized/answer/other/11800-0-0.sol
stackflow_sol_categorized/answer/other/11800-0-0.sol
stackflow_sol_categorized/answer/other/11800-0-0.sol
stackflow_sol_categorized/answer/other/28239-0-4.sol
stackflow_sol_categorized/answer/other/39259-0-0.sol
stackflow_sol_categorized/answer/other/39259-0-0.sol
stackflow_sol_categorized/answer/other/15385-0-0.sol
stackflow_sol_categorized/answer/other/54512-0-0.sol
stackflow_sol_categorized/answer/other/70035-0-1.sol
stackflow_sol_categorized/answer/other/63294-1-0.sol
stackflow_sol_categorized/answer/other/42091-0

7161